# Assembly: tracing assembly code (continued)
_COSC 208, Introduction to Computer Systems, 2024-03-27_

## Announcements
* Project 2 due today @ 11pm

## Outline
* Tracing assembly code

## No warm-up – Happy Wednesday!

## Tracing assembly code

* Example assembly code

```
0000000000000854 <deref>:
 854:   d10083ff    sub sp, sp, #0x20
 858:   f90007e0    str x0, [sp, #8]
 85c:   f94007e0    ldr x0, [sp, #8]
 860:   f9400000    ldr x0, [x0]
 864:   f9000fe0    str x0, [sp, #24]
 868:   f9400fe0    ldr x0, [sp, #24]
 86c:   910083ff    add sp, sp, #0x20
 870:   d65f03c0    ret
```

* Contents of the stack and registers after executing the first assembly instruction
    ```
    pc = 0x858
    sp = 0xF90
    w/x0 = 0xFB8
    ```
    ```
           0  1  2  3  4  5  6  7  8
     sp -> +--+--+--+--+--+--+--+--+
     0xF90 |                       |
           +--+--+--+--+--+--+--+--+
     0xF98 |                       |
           +--+--+--+--+--+--+--+--+
     0xFA0 |                       |
           +--+--+--+--+--+--+--+--+
     0xFA8 |                       |
           +--+--+--+--+--+--+--+--+
     0xFB0 |                       |
           +--+--+--+--+--+--+--+--+
     0xFB8 |           2           |
           +--+--+--+--+--+--+--+--+
     0xFC0 |         0x770         |
           +--+--+--+--+--+--+--+--+
     0xFC8 |                       |
           +--+--+--+--+--+--+--+--+
    ```

* Contents of the stack and registers *before* executing the second-to-last assembly instruction (`add sp, sp, #0x20`)
    ```
    pc = 0x86c
    sp = 0xF90
    w/x0 = 2
    ```
    ```
           0  1  2  3  4  5  6  7  8
     sp -> +--+--+--+--+--+--+--+--+
     0xF90 |                       |
           +--+--+--+--+--+--+--+--+
     0xF98 |         0xFB8         |
           +--+--+--+--+--+--+--+--+
     0xFA0 |           2           |
           +--+--+--+--+--+--+--+--+
     0xFA8 |                       |
           +--+--+--+--+--+--+--+--+
     0xFB0 |                       |
           +--+--+--+--+--+--+--+--+
     0xFB8 |           2           |
           +--+--+--+--+--+--+--+--+
     0xFC0 |         0x770         |
           +--+--+--+--+--+--+--+--+
     0xFC8 |                       |
           +--+--+--+--+--+--+--+--+
    ```

<div style="page-break-after:always;"></div>

_The following C code was compiled into assembly (using `gcc`)._

In [ ]:
/*1*/  int divide(unsigned int numerator, unsigned int denominator) {
/*2*/      int result = -1;
/*3*/      result = numerator / denominator;
/*4*/      return result;
/*5*/  }

Q1: _For each line of assembly, indicate which original line of C code the assembly instruction was derived from._

```
000000000000076c <divide>:                      // Line
    76c:    d10083ff     sub    sp, sp, #0x20   // 1
    770:    b9000fe0     str    w0, [sp, #12]   // 1
    774:    b9000be1     str    w1, [sp, #8]    // 1
    778:    12800000     mov    w0, #0xffffffff // 2
    77c:    b9001fe0     str    w0, [sp, #28]   // 2
    780:    b9400fe1     ldr    w1, [sp, #12]   // 3
    784:    b9400be0     ldr    w0, [sp, #8]    // 3
    788:    1ac00820     udiv   w0, w1, w0      // 3
    78c:    b9001fe0     str    w0, [sp, #28]   // 3
    790:    b9401fe0     ldr    w0, [sp, #28]   // 4
    794:    910083ff     add    sp, sp, #0x20   // 4
    798:    d65f03c0     ret                    // 4
```

```
000000000000076c <divide>:
    76c:    d10083ff     sub    sp, sp, #0x20
    770:    b9000fe0     str    w0, [sp, #12]
    774:    b9000be1     str    w1, [sp, #8]
    778:    12800000     mov    w0, #0xffffffff
    77c:    b9001fe0     str    w0, [sp, #28]
    780:    b9400fe1     ldr    w1, [sp, #12]
    784:    b9400be0     ldr    w0, [sp, #8]
    788:    1ac00820     udiv   w0, w1, w0
    78c:    b9001fe0     str    w0, [sp, #28]
    790:    b9401fe0     ldr    w0, [sp, #28]
    794:    910083ff     add    sp, sp, #0x20
    798:    d65f03c0     ret
```

Q2: _Assume the registers initially hold the following values:_
```
pc = 0x76C
sp = 0xFE0
w/x0 = 100
w/x1 = 5
```
_Draw the contents of the stack and registers after executing the assembly instruction at 0x790._

```
        0  1  2  3  4  5  6  7  8          
 sp ->  +--+--+--+--+--+--+--+--+
 0xFC0  |                       |
        +--+--+--+--+--+--+--+--+
 0xFC8  |     5     |    100    |
        +--+--+--+--+--+--+--+--+
 0xFD0  |                       |
        +--+--+--+--+--+--+--+--+
 0xFD8  |           |     20    |
        +--+--+--+--+--+--+--+--+
pc = 0x794
sp = 0xFC0
w/x0 = 20
w/x1 = 100
```

* _Why is `#0xffffffff` being stored in `w0`?_ — this is the two's complement representation of -1
* _When might this function cause an error?_ — when denominator is 0
* _How would you modify the C code to avoid an error?_ – add an if statement to verify that `denominator != 0`

<div style="page-break-after:always;"></div>

## Mapping C conditionals to assembly

In [ ]:
/*1*/  int divide_safe(unsigned int numerator, unsigned int denominator) {
/*2*/      int result = -1;
/*3*/      if (denominator != 0) {
/*4*/          result = numerator / denominator;
/*5*/      }
/*6*/      return result;
/*7*/  }

_Postponed to next class_

```
000000000000076c <divide_safe>:
    76c:    d10083ff     sub    sp, sp, #0x20
    770:    b9000fe0     str    w0, [sp, #12]
    774:    b9000be1     str    w1, [sp, #8]
    778:    12800000     mov    w0, #0xffffffff
    77c:    b9001fe0     str    w0, [sp, #28]
    780:    b9400be0     ldr    w0, [sp, #8]
    784:    7100001f     cmp    w0, #0x0
    788:    540000a0     b.eq   79c <divide_safe+0x30>
    78c:    b9400fe1     ldr    w1, [sp, #12]
    790:    b9400be0     ldr    w0, [sp, #8]
    794:    1ac00c20     udiv   w0, w1, w0
    798:    b9001fe0     str    w0, [sp, #28]
    79c:    b9401fe0     ldr    w0, [sp, #28]
    7a0:    910083ff     add    sp, sp, #0x20
    7a4:    d65f03c0     ret
```

<div style="page-break-after:always;"></div>

## Extra practice

Q3: _The following assembly code was generated (using `gcc`) for a C function with the prototype `int years_to_double(int rate)`:_

```
000000000000076c <years_to_double>:
    76c:    d10083ff    sub  sp, sp, #0x20
    770:    b9000fe0    str  w0, [sp, #12]
    774:    52800900    mov  w0, #0x48
    778:    b9001be0    str  w0, [sp, #24]
    77c:    b9401be1    ldr  w1, [sp, #24]
    780:    b9400fe0    ldr  w0, [sp, #12]
    784:    1ac00c20    sdiv w0, w1, w0
    788:    b9001fe0    str  w0, [sp, #28]
    78c:    b9401fe0    ldr  w0, [sp, #28]
    790:    910083ff    add  sp, sp, #0x20 
    794:    d65f03c0    ret
```

_Assume the registers initially hold the following values:_
```
pc = 0x76C
sp = 0x780
w/x0 = 9
```
_Draw the contents of the stack and registers after executing the assembly instruction at 0x78C._

```
        0  1  2  3  4  5  6  7  8          
 sp ->  +--+--+--+--+--+--+--+--+
 0x760  |                       |
        +--+--+--+--+--+--+--+--+
 0x768  |           |     9     |
        +--+--+--+--+--+--+--+--+
 0x770  |                       |
        +--+--+--+--+--+--+--+--+
 0x778  |    72     |     8     |
        +--+--+--+--+--+--+--+--+
pc = 0x790
sp = 0x760
w/x0 = 8
w/x1 = 72
```

<div style="page-break-after:always;"></div>

Q4: _The following assembly code was generated (using `gcc`) for a C function with the prototype `int sum(int a, int *b)`:_

```
0000000000000584 <sum>:
  584:    d10083ff     sub sp, sp, #0x20
  588:    b9001fe0     str w0, [sp, #28]
  58c:    f9000be1     str x1, [sp, #16]
  590:    f9400be8     ldr x8, [sp, #16]
  594:    b9400109     ldr w9, [x8]
  598:    b9000fe9     str w9, [sp, #12]
  59c:    b9401fe9     ldr w9, [sp, #28]
  5a0:    b9400fea     ldr w10, [sp, #12]
  5a4:    0b0a0129     add w9, w9, w10
  5a8:    b9000be9     str w9, [sp, #8]
  5ac:    b9400be0     ldr w0, [sp, #8]
  5b0:    910083ff     add sp, sp, #0x20
  5b4:    d65f03c0     ret
```

_Assume the registers and stack initially hold the following values:_

```
pc = 0x584
sp = 0xFB0
w/x0 = 2
w/x1 = 0xFC8 
```
```

















 sp -> +--+--+--+--+--+--+--+--+
 0xFB0 |                       |
       +--+--+--+--+--+--+--+--+
 0xFB8 |                       |
       +--+--+--+--+--+--+--+--+
 0xFC0 |         0xFC8         |
       +--+--+--+--+--+--+--+--+
 0xFC8 |     3     |     2     |
       +--+--+--+--+--+--+--+--+
 0xFD0 |         0x618         |
       +--+--+--+--+--+--+--+--+
 0xFD8 |                       |
       +--+--+--+--+--+--+--+--+
```
    
_Trace through the assembly code and update the contents of the stack and registers until the pc = 0x5B0._

```
pc = 0x5B0
sp = 0xF90
w/x0 = 5
w/x1 = 0xFC8
w/x8 = 0xFC8
w/x9 = 5
w/x10 = 3
```
```
       0  1  2  3  4  5  6  7  8
 sp -> +--+--+--+--+--+--+--+--+
 0xF90 |                       |
       +--+--+--+--+--+--+--+--+
 0xF98 |     5     |     3     |
       +--+--+--+--+--+--+--+--+
 0xFA0 |         0xFC8         |
       +--+--+--+--+--+--+--+--+
 0xFA8 |           |     2     |
       +--+--+--+--+--+--+--+--+
 0xFB0 |                       |
       +--+--+--+--+--+--+--+--+
 0xFB8 |                       |
       +--+--+--+--+--+--+--+--+
 0xFC0 |         0xFC8         |
       +--+--+--+--+--+--+--+--+
 0xFC8 |     3     |     2     |
       +--+--+--+--+--+--+--+--+
 0xFD0 |         0x618         |
       +--+--+--+--+--+--+--+--+
 0xFD8 |                       |
       +--+--+--+--+--+--+--+--+
```